# Reading data

In [9]:
import pandas as pd
import time
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
data = pd.read_csv('project.csv')
texts = data['texts'][:292]
labels = data['labels'][:292]

In [10]:
# Indecies used for demostration
demo_idx = [20, 50, 108, 83, 15, 33, 93, 120]

# Demo prompts for flipping label, input perturbation, complementary explanation

In [3]:
from demos import flipping, original, perturbation, complementary

In [22]:
import openai,os,sys

def chatgpt(texts, api_key, demo_prompts):
    openai.api_key = api_key
    chatgpt_answers = []

    for i in range(0, len(texts)):
        message = {demo_prompts + 'Review:' + texts[i] + '\nlabel:'}
        try:
            chat_completion = openai.ChatCompletion.create(
                        model="gpt-3.5-turbo",
                        messages=[message]
            )
            answer = chat_completion.choices[0].message.content
            chatgpt_answers.append(answer)
            # print(f"ChatGPT: {answer}")
        except:
            time.sleep(75)
            chat_completion = openai.ChatCompletion.create(
                        model="gpt-3.5-turbo",
                        messages=[message]
            )
            answer = chat_completion.choices[0].message.content
            chatgpt_answers.append(answer)

    return chatgpt_answers 

In [23]:
def gpt3(texts, api_key, demo_prompts):
    openai.api_key = api_key
    openai.Engine.list()
    gpt3_answers = []
    for i in range(0, len(texts)):
        message = {demo_prompts + 'Review:' + texts[i] + '\nlabel:'}
        
        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            temperature=0.1,
            max_tokens=50,
            top_p=1,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            # stop=["\n"]
          )
        print(response['choices'][0]['text'])
        gpt3_answers.append(response['choices'][0]['text'].replace('\n', '').replace(' ', ''))
    # model_answers.append(response)
    return response

In [24]:
def bert(texts, api_key, demo_prompts):
    # Load a pre-trained BERT model and tokenizer
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Create a sentiment analysis pipeline
    sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
    
    bert_answer = []
    for i in range(0, len(texts)):
        message = demo_prompts + 'Review:' + texts[i] + '\nlabel:'
        result = sentiment_pipeline(message)
        sentiment = result[0]["label"].lower()
        bert_answer.append(sentiment)
    
    return bert_answer

In [26]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
def roberta(texts, api_key, demo_prompts):
    model_name = "textattack/roberta-base-SST-2"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
    
    roberta_answer = []
    for i in range(0, len(texts)):
        message = demo_prompts + 'Review:' + texts[i] + '\nlabel:'
        result = sentiment_pipeline(message)
        sentiment = result[0]["label"].lower()
        if sentiment == 'label_1':
            roberta_answer.append('negative')
        else:
            roberta_answer.append('positive')
    
    return roberta_answer

# Calculate results for different input prompts and models

In [27]:
API_KEY = 'your api key'
input_prompts = [original, flipping, perturbation, complementary]
input_keys = ['original', 'flipping', 'perturbation', 'complementary']

project_data = {
    'texts':texts,
    'labels':labels
}

for i, prompt in enumerate(input_prompts):
    chatgpt_result = chatgpt(texts, API_KEY, prompt)
    gpt3_result = gpt3(texts, API_KEY, prompt)
    bert_result = bert(texts, API_KEY, prompt)
    roberta_result = roberta(texts, API_KEY, prompt)
    
    project_data['chatgpt_'+input_keys[i]] = chatgpt_result
    project_data['gpt3_'+input_keys[i]] = gpt3_result
    bert_data['bert_'+input_keys[i]] = bert_result
    roberta_data['roberta_'+input_keys[i]] = roberta_result
    


df = pd.DataFrame(project_data)
df.to_csv('model_answers.csv', index=False)

KeyboardInterrupt: 